In [1]:
import pandas as pd
import sqlite3 as sql3
from time import sleep
import matplotlib.pyplot as plt
import numpy as np

In [2]:
%matplotlib inline
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

In [3]:

def generetPie(serie):
    
    texto, fatias = plt.pie(serie.value_counts(), startangle=90  )
    plt.legend(texto,serie.value_counts().index,bbox_to_anchor = (1.25, 1))



#Função para gerar cores aletórias
def generetColor(num):
    listaHSV = [(x*1.0/num, 0.5, 0.5) for x in range(num)]
    return list(map(lambda x: colorsys.hsv_to_rgb(*x), listaHSV))



In [4]:
conn = sql3.connect('PJe-TRF.sqlite3/PJe-TRF.sqlite3')

In [5]:
qtdLinhas = 100

# Lendo os dados das decisões

In [53]:
decisoes = pd.read_sql('''
    SELECT * FROM decisoes limit {}
'''.format(qtdLinhas), con=conn)

In [55]:
decisoes.head()

,num_processo,documento,html
0,0819306-76.2017.4.05.8300,31/08/2018 14:04:40 - Inteiro Teor - Inteiro T...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he..."
1,0819306-76.2017.4.05.8300,31/08/2018 14:04:40 - Inteiro Teor - Inteiro T...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he..."
2,0819306-76.2017.4.05.8300,13/06/2018 12:39:16 - Sentença - Sentença,"<html xmlns=""http://www.w3.org/1999/xhtml""><he..."
3,0819306-76.2017.4.05.8300,21/05/2018 17:50:07 - Sentença - Sentença,"<html xmlns=""http://www.w3.org/1999/xhtml""><he..."
4,0819306-76.2017.4.05.8300,15/02/2018 16:12:07 - Decisão - Decisão,"<html xmlns=""http://www.w3.org/1999/xhtml""><he..."


In [56]:
# separando os dados das decisões em duas colunas
decisoes['data'] = ''
for index, row in decisoes.iterrows():
    string = decisoes.iloc[index]['documento'] 
    pos = string.find('-')
    decisoes.iloc[index]['data'] = string[0:pos]
    decisoes.iloc[index]['documento'] = string[pos+1 :]    

In [59]:
decisoes.head() 

,num_processo,documento,html,data
0,0819306-76.2017.4.05.8300,Inteiro Teor - Inteiro Teor do Acórdão,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",31/08/2018 14:04:40
1,0819306-76.2017.4.05.8300,Inteiro Teor - Inteiro Teor do Acórdão,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",31/08/2018 14:04:40
2,0819306-76.2017.4.05.8300,Sentença - Sentença,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",13/06/2018 12:39:16
3,0819306-76.2017.4.05.8300,Sentença - Sentença,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",21/05/2018 17:50:07
4,0819306-76.2017.4.05.8300,Decisão - Decisão,"<html xmlns=""http://www.w3.org/1999/xhtml""><he...",15/02/2018 16:12:07


In [86]:
movimentacoes = pd.read_sql('''
    SELECT * FROM movimentacoes limit {}
'''.format(qtdLinhas), con=conn)

In [87]:
movimentacoes.head()

,movimento,documento,aplicacao,num_processo
0,30/11/2018 11:11:37 - Juntada de Recurso Extra...,None,2º Grau,0819306-76.2017.4.05.8300
1,30/11/2018 11:10:18 - Juntada de Recurso Especial,None,2º Grau,0819306-76.2017.4.05.8300
2,25/11/2018 00:01:33 - Juntada de Certidão de I...,25/11/2018 00:01:33 - Certidão de Intimação - ...,2º Grau,0819306-76.2017.4.05.8300
3,19/11/2018 10:29:49 - Juntada de Certidão de I...,19/11/2018 10:29:48 - Certidão de Intimação - ...,2º Grau,0819306-76.2017.4.05.8300
4,14/11/2018 19:46:51 - Expedição de expediente,None,2º Grau,0819306-76.2017.4.05.8300


In [88]:
# separando os dados das decisões em duas colunas
movimentacoes['data_movimento'] = ''
for index, row in movimentacoes.iterrows():
    string = movimentacoes.iloc[index]['movimento'] 
    pos = string.find('-')
    movimentacoes.iloc[index]['data_movimento'] = string[0:pos]
    movimentacoes.iloc[index]['movimento'] = string[pos+1 :]    

In [89]:
movimentacoes.head()

,movimento,documento,aplicacao,num_processo,data_movimento
0,Juntada de Recurso Extraordinário,None,2º Grau,0819306-76.2017.4.05.8300,30/11/2018 11:11:37
1,Juntada de Recurso Especial,None,2º Grau,0819306-76.2017.4.05.8300,30/11/2018 11:10:18
2,Juntada de Certidão de Intimação,25/11/2018 00:01:33 - Certidão de Intimação - ...,2º Grau,0819306-76.2017.4.05.8300,25/11/2018 00:01:33
3,Juntada de Certidão de Intimação,19/11/2018 10:29:48 - Certidão de Intimação - ...,2º Grau,0819306-76.2017.4.05.8300,19/11/2018 10:29:49
4,Expedição de expediente,None,2º Grau,0819306-76.2017.4.05.8300,14/11/2018 19:46:51


In [90]:
# separando os dados das decisões em duas colunas
movimentacoes['data_documento'] = ''
for index, row in movimentacoes.iterrows():
    try:
        string = movimentacoes.iloc[index]['documento'] 
        pos = string.find('-')
        movimentacoes.iloc[index]['data_documento'] = string[0:pos]
        movimentacoes.iloc[index]['documento'] = string[pos+1 :]    
    except:
        continue

In [92]:
movimentacoes.head(100)

,movimento,documento,aplicacao,num_processo,data_movimento,data_documento
0,Juntada de Recurso Extraordinário,None,2º Grau,0819306-76.2017.4.05.8300,30/11/2018 11:11:37,
1,Juntada de Recurso Especial,None,2º Grau,0819306-76.2017.4.05.8300,30/11/2018 11:10:18,
2,Juntada de Certidão de Intimação,Certidão de Intimação - Certidão de Intimação,2º Grau,0819306-76.2017.4.05.8300,25/11/2018 00:01:33,25/11/2018 00:01:33
3,Juntada de Certidão de Intimação,Certidão de Intimação - Certidão de Intimação,2º Grau,0819306-76.2017.4.05.8300,19/11/2018 10:29:49,19/11/2018 10:29:48
4,Expedição de expediente,None,2º Grau,0819306-76.2017.4.05.8300,14/11/2018 19:46:51,
5,Deliberado em Sessão - Julgado,None,2º Grau,0819306-76.2017.4.05.8300,14/11/2018 17:10:04,
6,Juntada de Certidão,Certidão de Julgamento - Certidão,2º Grau,0819306-76.2017.4.05.8300,14/11/2018 16:58:26,14/11/2018 16:58:25
7,Juntada de Certidão de Intimação,Certidão de Intimação - Certidão de Intimação,2º Grau,0819306-76.2017.4.05.8300,02/11/2018 15:32:06,02/11/2018 15:32:06
8,Juntada de Certidão de Intimação,Certidão de Intimação - Certidão de Intimação,2º Grau,0819306-76.2017.4.05.8300,01/11/2018 11:44:39,01/11/2018 11:44:39
9,Juntada de Certidão de Intimação,Certidão de Intimação - Certidão de Intimação,2º Grau,0819306-76.2017.4.05.8300,01/11/2018 11:39:12,01/11/2018 11:39:12


In [93]:
processos = pd.read_sql('''
                        Select * from processos limit {}
                        '''.format(qtdLinhas), con = conn)

In [97]:
processos.head()

,num_processo,polo_ativo,polo_passivo,data_distrib,orgao_julgador,classe_judicial,assunto,assuntos_tabela,url_atual,fonte_pagina
0,0819306-76.2017.4.05.8300,JOSE EDSON DA SILVA LINS\nAPELANTE\n JOAO L...,INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS\nAP...,27/07/2018,Gab 15 - Des. EDILSON NOBRE,APELAÇÃO CÍVEL,DIREITO PREVIDENCIÁRIO|Benefícios em Espécie|A...,DIREITO PREVIDENCIÁRIO|Benefícios em Espécie|A...,https://pje.trf5.jus.br/pje/ConsultaPublica/De...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he..."
1,0819287-70.2017.4.05.8300,INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS\nAP...,TOMAS DE ALBUQUERQUE LAPA\nAPELADO\n RICARD...,23/04/2018,Gab 2 - Des. ALEXANDRE LUNA FREIRE,APELAÇÃO CÍVEL,DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...,DIREITO ADMINISTRATIVO E OUTRAS MATÉRIAS DE DI...,https://pje.trf5.jus.br/pje/ConsultaPublica/De...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he..."
2,0819272-04.2017.4.05.8300,SHEILA MARIA VIEIRA DE ALMEIDA\nAPELANTE\n ...,INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS\nAP...,21/11/2018,Gab 12 - Des. VLADIMIR CARVALHO,APELAÇÃO CÍVEL,DIREITO PREVIDENCIÁRIO|Benefícios em Espécie|A...,DIREITO PREVIDENCIÁRIO|Benefícios em Espécie|A...,https://pje.trf5.jus.br/pje/ConsultaPublica/De...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he..."
3,0819217-53.2017.4.05.8300,KLEBER FERREIRA DA SILVA\nAPELANTE\n GENER ...,INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS\nAP...,24/10/2018,Gab 2 - Des. ALEXANDRE LUNA FREIRE,APELAÇÃO CÍVEL,DIREITO PREVIDENCIÁRIO|Benefícios em Espécie|P...,DIREITO PREVIDENCIÁRIO|Benefícios em Espécie|P...,https://pje.trf5.jus.br/pje/ConsultaPublica/De...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he..."
4,0819216-68.2017.4.05.8300,INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS\nAP...,ANTONIO ALVES TIMOTEO JUNIOR\nAPELADO\n luc...,09/08/2018,Gab 1 - Des. LÁZARO GUIMARÃES,APELAÇÃO CÍVEL,DIREITO PREVIDENCIÁRIO|Benefícios em Espécie|A...,DIREITO PREVIDENCIÁRIO|Benefícios em Espécie|A...,https://pje.trf5.jus.br/pje/ConsultaPublica/De...,"<html xmlns=""http://www.w3.org/1999/xhtml""><he..."


In [98]:
processos.polo_passivo.

INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS\nAPELADO                                                                                                                                                              33
CARMELITA DOS SANTOS CARVALHO\nAPELADO\n    MARCOS ANTONIO MENEZES PRADO - SE4485\nADVOGADO                                                                                                                       2
ELIANE DE JESUS SOUZA\nAPELADO\n    AIRTON CISNEIROS DOS SANTOS - SE5149\nADVOGADO                                                                                                                                2
SINEZIO SOARES DE OLIVEIRA FILHO\nAPELADO\n    JOÃO BATISTA DE FREITAS - PE012112-D\nADVOGADO                                                                                                                     2
GILSON BATISTA DA SILVA\nAPELADO\n    Ricardo de Souza - PE19444\nADVOGADO\n    Sofia Ferreira Alencar Correa Debassolles - PE040571-A\nADVOGADO        

In [100]:
processos.polo_ativo.value_counts()

INSTITUTO NACIONAL DO SEGURO SOCIAL - INSS\nAPELANTE                                                                                                                                                                                                                               52
MARIA DO SOCORRO CARNEIRO HENRIQUE\nAPELANTE\n    NATHALIE DA NOBREGA MEDEIROS - PB17190\nADVOGADO\nNão validado                                                                                                                                                                    2
JAILDO SANTOS VIANA\nAPELANTE\n    nadieje wanderley de siqueira - PE20055\nADVOGADO\nNão validado\n    JOAQUIM PEDRO CARNEIRO CAMPELLO FILHO - PE36681\nADVOGADO                                                                                                                   2
SANDRA MOURA DE OLIVEIRA\nAPELANTE\n    Elismar Santos - SE4676-A\nADVOGADO                                                                                           